In [1]:
!pip install pandas
!pip install numpy
!pip install re
!pip install os
!pip install matplotlib
!pip install statsmodels
!pip install seaborn
!pip install scipy.stats
!pip install scikit_posthocs

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


ERROR: Could not find a version that satisfies the requirement scipy.stats (from versions: none)
ERROR: No matching distribution found for scipy.stats


In [3]:
import pandas as pd
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene, shapiro, kruskal
import numpy as np
from scipy.stats import sem, t

# Define the Kruskal-Wallis function.
def process_kruskal(input_file, output_kruskal):
    # Load the data.
    data = pd.read_csv(input_file)

    # Open output files once at the start
    kruskal_output = open(output_kruskal, 'w')

    # Process data by rank.
    for rank in data['rank'].unique():
        # Filter data for the current rank.
        rank_data = data[data['rank'] == rank]

        # Group the data by architecture for Kruskal-Wallis
        grouped_data = [group['absorbedPAR_umol_m2_s1'].values for name, group in rank_data.groupby('architecture')]

        # Perform Kruskal-Wallis test
        kruskal_stat, kruskal_p = kruskal(*grouped_data)

        # Function to format p-values with a minimum display value of 0.001
        def format_p_value(p):
            if p < 0.001:
                return "<0.001"
            else:
                return round(p, 3)

        kruskal_p_formatted = format_p_value(kruskal_p)

        # Write Kruskal-Wallis results for this rank to the file
        kruskal_output.write(f"\nKruskal-Wallis test results for rank {rank}:\n")
        kruskal_output.write(f'Statistic={kruskal_stat}, p-value={kruskal_p_formatted}\n')

        # Post-hoc test (Dunn’s test).
        if kruskal_p < 0.05:  # Check if the overall model is significant.
            posthoc = sp.posthoc_dunn(rank_data, val_col='absorbedPAR_umol_m2_s1', group_col='architecture', p_adjust='bonferroni')
            posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
            kruskal_output.write(f'\nPost-hoc (Dunn) Results:\n{posthoc}\n')

    # Close the output files after all ranks are processed
    kruskal_output.close()


# Define input files and corresponding output paths
files = [
    {
        'input': 'combined_files/combined_high_ranks_cleaned.csv',
        'output_kruskal': 'output_statistics/Kruskal_high_ranks.txt',
    },
    {
        'input': 'combined_files/combined_low_ranks_cleaned.csv',
        'output_kruskal': 'output_statistics/Kruskal_low_ranks.txt',
    }
]

# Loop through each file and apply the Kruskal-Wallis function
for file_info in files:
    process_kruskal(file_info['input'], file_info['output_kruskal'])

print("Kruskal-Wallis processing complete for all files.")


Kruskal-Wallis processing complete for all files.


C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")


In [4]:
import pandas as pd
import scikit_posthocs as sp
import matplotlib.pyplot as plt
from scipy.stats import levene, shapiro, kruskal
import seaborn as sns

# Define a function to process Kruskal-Wallis, assumptions, and plotting for each dataset.
def process_data(input_file, output_kruskal, output_assumptions, qqplot_path):
    # Load the data.
    data = pd.read_csv(input_file)
    print(data)

    # Create output.txt for Kruskal-Wallis results and assumptions.
    kruskal_output = open(output_kruskal, 'w')

    # Kruskal-Wallis test.
    grouped_data = [group['absorbedPAR_umol_m2_s1'].values for name, group in data.groupby('architecture')]
    kruskal_stat, kruskal_p = kruskal(*grouped_data)

    # Function to format p-values with a minimum display value of 0.001
    def format_p_value(p):
        if p < 0.001:
            return "<0.001"
        else:
            return round(p, 3)

    kruskal_p_formatted = format_p_value(kruskal_p)

    # Write Kruskal-Wallis results to the file
    kruskal_output.write(f"\nKruskal-Wallis test results for total absorbed PAR:\n")
    kruskal_output.write(f'Statistic={kruskal_stat}, p-value={kruskal_p_formatted}\n')

    # Post-hoc test (Dunn’s test).
    if kruskal_p < 0.05:  # Check if the overall test is significant.
        posthoc = sp.posthoc_dunn(data, val_col='absorbedPAR_umol_m2_s1', group_col='architecture', p_adjust='bonferroni')
        posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
        kruskal_output.write(f'\nPost-hoc (Dunn) Results:\n{posthoc}\n')

    # Close output files.
    kruskal_output.close()


# Define input files and corresponding output paths for high and low densities.
files = [
    {
        'input_file': 'combined_files/combined_total_absorbedPAR_high_cleaned.csv',
        'output_kruskal': 'output_statistics/Kruskal_total_absorbedPAR_high.txt',
        'output_assumptions': 'output_statistics/assumptions_total_absorbedPAR_high.txt',
        'qqplot_path': 'qq_plots/total_absorbedPAR_high.png',
    },
    {
        'input_file': 'combined_files/combined_total_absorbedPAR_low_cleaned.csv',
        'output_kruskal': 'output_statistics/Kruskal_total_absorbedPAR_low.txt',
        'output_assumptions': 'output_statistics/assumptions_total_absorbedPAR_low.txt',
        'qqplot_path': 'qq_plots/total_absorbedPAR_low.png',
    },
    {
        'input_file': 'combined_files/combined_high_sensors_cleaned.csv',
        'output_kruskal': 'output_statistics/Kruskal_sensors_high.txt',
        'output_assumptions': 'output_statistics/assumptions_sensors_high.txt',
        'qqplot_path': 'qq_plots/sensors_high.png',
    },
    {
        'input_file': 'combined_files/combined_low_sensors_cleaned.csv',
        'output_kruskal': 'output_statistics/Kruskal_sensors_low.txt',
        'output_assumptions': 'output_statistics/assumptions_sensors_low.txt',
        'qqplot_path': 'qq_plots/sensors_low.png',
    }
]

# Loop through the files and apply the function to each file
for file_info in files:
    process_data(
        input_file=file_info['input_file'],
        output_kruskal=file_info['output_kruskal'],
        output_assumptions=file_info['output_assumptions'],
        qqplot_path=file_info['qqplot_path'],
    )

print("Kruskal-Wallis analysis completed for all datasets.")


    density   architecture  repetition  absorbedPAR_umol_m2_s1  area_m2[m^2]
0      High  architectureA           1              886.626033      0.050084
1      High  architectureA           2              898.142581      0.050102
2      High  architectureA           3              841.565563      0.050074
3      High  architectureA           4              864.056255      0.050066
4      High  architectureA           5              838.677274      0.050043
..      ...            ...         ...                     ...           ...
493    High        control          96              827.087922      0.050158
494    High        control          97              797.928454      0.050116
495    High        control          98              858.325426      0.050052
496    High        control          99              801.242413      0.050085
497    High        control         100              830.831216      0.049989

[498 rows x 5 columns]
    density   architecture  repetition  absorbedPAR_

C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\1790449089.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\1790449089.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\1790449089.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_5668\1790449089.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
